<a href="https://colab.research.google.com/github/GaohuiXu/llm-cookbook/blob/main/content/%E5%BF%85%E4%BF%AE%E4%B8%80-Prompt%20Engineering%20For%20Developers/2.%20%E6%8F%90%E7%A4%BA%E5%8E%9F%E5%88%99%20Guidelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第二章 编写 Prompt 的原则

  本章的主要内容为编写 Prompt 的原则，在本章中，我们将给出两个编写 Prompt 的原则与一些相关的策略，您可以练习编写高效的 Prompt，从而便捷而有效地使用 LLM。

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一环境配置" data-toc-modified-id="一、环境配置">一、环境配置</a></span></li>
        <li>
            <span><a href="#二两个基本原则" data-toc-modified-id="二、两个基本原则">二、两个基本原则</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-原则一编写清晰具体的指令" data-toc-modified-id="2.1 原则一：编写清晰、具体的指令">2.1 原则一：编写清晰、具体的指令</a></span></li>
                <li><span><a href="#22-给模型时间去思考" data-toc-modified-id="2.2 原则二：给模型时间去思考">2.2 原则二：给模型时间去思考</a></span></li>
            </ul>
        </li>
        <li><span><a href="#三局限性" data-toc-modified-id="三、局限性">三、局限性</a></span>
        </li>
    </ul>
</div>

## 一、环境配置

本教程使用 OpenAI 所开放的 ChatGPT API，因此您需要首先拥有一个 ChatGPT 的 API_KEY（也可以直接访问官方网址在线测试），然后需要安装 OpenAI 的第三方库。为了兼顾简便与兼容性，本教程将介绍在 ```Python 3``` 环境中基于 ```openai.api_key``` 方法的配置。另有基于环境变量的配置方法，详情请参考 [OpenAI 官方文档](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)。

首先需要安装 OpenAI 库：
```bash
pip install openai
```

In [1]:
!pip install openai==0.28

In [18]:
import openai
# 导入第三方库

openai.api_key  = "sk-de071700bc094713b4567d255658606f"
openai.api_base = "https://api.deepseek.com/v1"
# 以下为基于环境变量的配置方法示例，这样更加安全。仅供参考，后续将不再涉及。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY


整个课程将以 gpt-3.5-turbo 模型为例。我们将在后续课程中深入探究 OpenAI 提供的 [Chat Completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api) 的使用方法，在此处，我们先将它封装成一个函数，您无需知道其内部机理，仅需知道调用该函数，以 Prompt 为输入参数，其将会输出对应的 Completion （回答结果）即可。

In [24]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model='deepseek-reasoner',
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、两个基本原则

### 2.1 原则一：编写清晰、具体的指令

您应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并降低您得到无关或不正确响应的可能性。清晰的指令不意味着必须简短，在许多情况下，更长的 Prompt 实际上更清晰，且提供了更多上下文，也就可能产生更详细更相关的输出。

**2.1.1 使用分隔符清晰地表示输入的不同部分**

分隔符可以是：```，""，<>，:，\<tag> \</tag>等。

您可以使用任何明显的标点符号将特定的文本部分与 Prompt 的其余部分分开。标记的形式不限，只需要让模型明确知道这是一个单独部分。使用分隔符可以有效避免提示词注入( Prompt injection )。提示词注入是指如果允许用户将某些输入添加到（开发者预定义的） Prompt 中，则所提供的指令可能会与开发者想要执行的操作相冲突，从而使 LLM 遵循用户输入的指令，而非执行开发者预期的操作。即，输入里面可能包含其他指令，会覆盖掉您的指令。对此，使用分隔符是一个不错的策略。

在以下的例子中，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符。


In [21]:
# 中文版见下一个 cell
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To achieve the desired output from a model, provide clear, specific, and detailed instructions, as longer prompts often offer better clarity and context than shorter ones, reducing irrelevant or incorrect responses.


Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.

In [22]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

清晰的提示词应具体详细，避免简短模糊，以确保模型输出准确相关。



提供清晰具体的指示，避免无关或不正确响应，不要混淆写清晰和写简短，更长的提示可以提供更多清晰度和上下文信息，导致更详细和相关的输出。

**2.1.2 寻求结构化的输出**

输出可以是 Json、HTML 等格式。

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，您可以在 Python 中将其读入字典或列表中。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [25]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)


```json
[
  {
    "book_id": 1,
    "title": "The Whispering Sands of Al-Kharid",
    "author": "Elias Thornfield",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "Chronicles of the Celestial Nomad",
    "author": "Dr. Lila Voss",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Silent Key",
    "author": "Margo Blackthorn",
    "genre": "Mystery/Thriller"
  }
]
```


In [26]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)


```json
[
  {
    "book_id": 1,
    "title": "The Ashes of Stardust",
    "author": "Aria Voss",
    "genre": "Science Fiction"
  },
  {
    "book_id": 2,
    "title": "The Jade Labyrinth",
    "author": "Haruki Mizuno",
    "genre": "Fantasy"
  },
  {
    "book_id": 3,
    "title": "The Last Telephone Booth",
    "author": "Eleanor Whitlock",
    "genre": "Mystery"
  }
]
```


**2.1.3 要求模型检查是否满足条件**

如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指出并停止执行后续的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或错误发生。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答“未提供步骤”。

In [27]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.  
Step 2 - Grab a cup and put a tea bag in it while the water heats.  
Step 3 - Pour the hot water over the tea bag once it’s ready.  
Step 4 - Let the tea steep for a bit.  
Step 5 - Remove the tea bag after a few minutes.  
Step 6 - Add sugar or milk to taste (optional).


In [28]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:
Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


In [29]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。  
第二步 - 准备一个杯子并放入茶包。  
第三步 - 将烧开的水倒入装有茶包的杯子中。  
第四步 - 等待茶叶浸泡数分钟。  
第五步 - 取出茶包。  
第六步 - 根据个人喜好添加糖或牛奶调味。


In [ ]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


**2.1.4 提供少量示例**（少样本提示词，Few-shot prompting）

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和祖父之间的对话的例子。孩子说，“请教我何为耐心”，祖父用下述风格的隐喻来回答。由于我们已经告诉模型要以一致的语气回答，因此现在我们问“请教我何为韧性”，由于模型已经有了这个少样本示例( few-shot example )，它将以类似的语气回答下一个任务。

In [30]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The oak that shelters generations \  
once trembled in the fiercest gales; the steel \  
that bears the weight of bridges was forged \  
in the furnace’s unrelenting flame; the diamond \  
that dazzles the eye endured the dark’s embrace \  
for a thousand years.


In [31]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 孩子啊，风暴中屹立的橡树曾在无数狂风里学会扎根；竹子能弯至地面却从不折断，因其懂得随风起伏的智慧；最璀璨的钻石历经地底深处的重压方显光芒。韧性不是与风暴对抗，而是学会如何与它共舞。


### 2.2 给模型时间去思考

如果您发现模型推理过程过于匆忙，导致得出了错误的结论，那么您应该尝试重新构思 Prompt ，要求模型在提供最终答案之前开展**思维链**，或进行一系列相关推理（a chain or series of relevant reasoning）。换句话说，如果您给模型一个在短时间内或用少量文字无法完成的复杂任务，它的输出结果就容易出错。这种情况对人来说也是类似：如果您要求某人完成复杂的数学问题，又不给足够时间计算出答案，他们也可能会犯错误。因此，在这些情况下，您应该指示模型花更多时间思考问题，让它在任务上花费更多计算资源。

**2.2.1 指定完成任务所需的步骤**

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。

首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和人名个数。要求输出以换行符分隔。

In [32]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Siblings Jack and Jill's quest to fetch water from a hilltop well ends in a tumble down the hill, but they return home comforted and remain undeterred in their adventurous spirits.  

Frère et sœur, Jack et Jill, partent chercher de l'eau d'un puits sur une colline mais tombent en chemin, rentrent chez eux réconfortés et gardent leur esprit aventureux intact.  

Jack, Jill  

{  
  "french_summary": "Frère et sœur, Jack et Jill, partent chercher de l'eau d'un puits sur une colline mais tombent en chemin, rentrent chez eux réconfortés et gardent leur esprit aventureux intact.",  
  "num_names": 2  
}


In [ ]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1-兄妹在山顶井里打水时发生意外，但仍然保持冒险精神。
2-Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.
3-Jack, Jill
4-{
   "French_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.",
   "num_names": 2
}


上述输出仍然存在一定问题，例如，键“姓名”会被替换为法语（译注：在英文原版中，对应指令第三步的输出为 'Noms:',为Name的法语，这种行为难以预测，并可能为导出带来困难）

因此，我们将Prompt加以改进，该 Prompt 前半部分不变，同时**确切指定了输出的格式**。

In [33]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Siblings Jack and Jill's quest to fetch water from a hilltop well ends in a tumble down the hill, but their adventurous spirits remain unshaken.  
Translation: Dans un charmant village, les frères et sœurs Jack et Jill partent chercher de l'eau à un puits au sommet d'une colline, mais tombent en chemin, gardant malgré tout leur esprit aventureux intact.  
Names: ["Jack", "Jill"]  
Output JSON:  
```json  
{  
  "french_summary": "Dans un charmant village, les frères et sœurs Jack et Jill partent chercher de l'eau à un puits au sommet d'une colline, mais tombent en chemin, gardant malgré tout leur esprit aventureux intact.",  
  "num_names": 2  
}  
```


In [ ]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：兄妹杰克和吉尔在迷人的村庄里冒险，不幸摔伤后回到家中，但仍然充满冒险精神。
翻译：In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.
名称：Jack，Jill
输出 JSON：{"English_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.", "num_names": 2}


**2.2.2 指导模型在下结论之前找出一个自己的解法**

明确地指引模型在匆匆做决策之前，要自己思考出一份解决方案。有时这样会得到更好的结果。这与之前所述思想类似，即给模型时间思考。

接下来我们会给出一个问题和一份来自学生的解答，要求模型判断解答是否正确：

In [34]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is **incorrect**. 

**Error Identified:**  
The maintenance cost's variable component was miscalculated. The problem states maintenance is \$100k/year + \$10/square foot, but the student used \$100x instead of \$10x.

**Correct Calculation:**  
1. **Land:** \(100x\)  
2. **Solar Panels:** \(250x\)  
3. **Maintenance:** \(100,000 + 10x\)  

**Total Cost:**  
\[
100x + 250x + 10x + 100,000 = 360x + 100,000
\]  

**Final Answer:**  
The total first-year cost as a function of square feet is \( \boxed{360x + 100,\!000} \).


In [35]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案中存在错误。错误出现在维护费用的计算部分。

**逐步解释：**

1. **土地费用**：正确计算为 \(100x\) 美元。
2. **太阳能电池板费用**：正确计算为 \(250x\) 美元。
3. **维护费用**：
   - 固定费用：\(100,000\) 美元（正确）。
   - 额外费用应为 \(10x\) 美元（题目中明确为每平方英尺10美元），但学生错误地计算为 \(100x\) 美元。
4. **总费用计算**：
   - 正确公式应为：\(100x + 250x + 10x + 100,000 = 360x + 100,000\) 美元。
   - 学生错误地求和为 \(450x + 100,000\) 美元，因其将维护的额外费用多算了 \(90x\)。

**结论**：学生的答案不正确。正确首年总费用为 **\(360x + 100,000\) 美元**。


但是注意，学生的解决方案实际上是错误的。（*维护费用项100x应为10x，总费用450x应为360x*）

我们可以通过指导模型先自行找出一个解法来解决这个问题。

在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学生的解法是否正确。同时，我们给定了输出的格式要求。通过拆分任务、明确步骤，让模型有更多时间思考，有时可以获得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [36]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

```
The problem requires calculating the total first-year cost as a function of square feet (x). 

1. **Land Cost**: $100 per square foot → 100x
2. **Solar Panel Cost**: $250 per square foot → 250x
3. **Maintenance Cost**: Flat $100,000/year + $10 per square foot → 100,000 + 10x

Summing these:
Total Cost = 100x + 250x + 100,000 + 10x = (100+250+10)x + 100,000 = 360x + 100,000

The student incorrectly used 100x instead of 10x for the maintenance variable cost, leading to an overestimation.
```

Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


In [ ]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
**学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否**
学生的成绩：正确或不正确

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

正确的解决方案和步骤：
    1. 计算土地费用：100美元/平方英尺 * x平方英尺 = 100x美元
    2. 计算太阳能电池板费用：250美元/平方英尺 * x平方英尺 = 250x美元
    3. 计算维护费用：10万美元 + 10美元/平方英尺 * x平方英尺 = 10万美元 + 10x美元
    4. 计算总费用：100x美元 + 250x美元 + 10万美元 + 10x美元 = 360x + 10万美元

学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


## 三、局限性

**开发大模型相关应用时请务必铭记：**


**虚假知识**：模型偶尔会生成一些看似真实实则编造的知识

虽然模型在训练过程中接触了大量的知识，但它并没有*完全*记住所见的信息，因此它不甚清楚自己知识的边界。这意味着它可能会尝试回答主题晦涩难懂的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编造的想法为幻觉（Hallucination）。

如下示例展示了大模型的幻觉。我们要求告诉我们 Boie 公司生产的 *AeroGlide UltraSlim Smart Toothbrush* 产品的信息，事实上，这个公司是真实存在的，但产品是编造的，而模型一本正经地提供了它编造的知识，而且迷惑性很强。



In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its advanced sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the toughest stains and buildup, leaving your teeth feeling clean and fresh.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.

Overall, the AeroGlide UltraS

In [37]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

根据目前的公开信息，Boie 公司并未推出名为 **AeroGlide UltraSlim Smart Toothbrush** 的产品。以下是可能相关的背景信息和建议，供您参考：

---

### **关于 Boie 公司**
Boie 是一家以创新设计和可持续理念闻名的个人护理品牌，主要产品包括：
- **环保牙刷**：可更换刷头设计，刷毛采用耐用聚合物（非尼龙），刷柄由回收材料制成。
- **剃须刀等产品**：同样注重可重复使用性和环保材料。

其牙刷产品通常以 **极简设计** 和 **轻薄刷头** 为特点，但暂未涉及“智能”（如蓝牙连接、APP 追踪）功能。

---

### **可能混淆的情况**
1. **品牌混淆**：  
   - **Oral-B** 曾推出过 **AeroGlide** 系列牙刷（非智能款），主打超薄刷头。  
   - **飞利浦 Sonicare** 或 **Oral-B iO** 系列则包含智能牙刷产品（支持蓝牙与APP联动）。

2. **产品名称误差**：  
   Boie 的经典款牙刷名为 **Boie Curve Toothbrush**，具有超薄刷头，但无智能功能。

---

### **建议行动**
1. **核实产品名称或品牌**：  
   确认是否混淆了品牌或型号名称。您可通过 Boie 官网（[boieusa.com](https://www.boieusa.com/)）查看其最新产品线。

2. **替代产品推荐**：  
   - 若需 **环保牙刷**：Boie Curve Toothbrush 是理想选择。  
   - 若需 **智能牙刷**：可参考飞利浦 Sonicare DiamondClean 或 Oral-B iO 系列。

---

如需进一步帮助，请提供更多细节（如功能描述或产品图片），我将尽力协助！ 😊


由于很容易以假乱真，请读者根据在本系列教程中所学知识，在构建自己的应用程序时尽量避免幻觉情况。幻觉是大模型的一个已知缺陷（注：截至2023年7月），OpenAI也在努力解决该问题。

在您希望模型根据文本生成回答时，另一种减少幻觉的策略是先要求模型获取来源于该文本的所有引用信息（任何相关引用，any relevant quotes），然后要求它基于所引用的信息来回答问题，这使得我们能根据答案追溯源文档，通常对减少幻觉非常有帮助。

**关于反斜杠使用的说明：**

在本教程中，我们使用反斜杠 \ 来使文本适应屏幕大小以提高阅读体验，而没有用换行符 \n 。GPT-3 并不受换行符（newline characters）的影响，但在您调用其他大模型时，需额外考虑换行符是否会影响模型性能。